In [0]:
# Databricks Notebook: Transform Bronze (Futures Leaderboard Positions) -> Silver
from pyspark.sql.functions import col
from delta.tables import DeltaTable

CATALOG = "demo_catalog"
SCHEMA  = "demo_schema"
BRONZE  = f"{CATALOG}.{SCHEMA}.bronze_futures_leaderboard_positions"
SILVER  = f"{CATALOG}.{SCHEMA}.silver_futures_positions"

# =========================
# (A) Silver 테이블 생성
# =========================
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {SILVER} (
  uid        STRING,
  symbol     STRING,
  entryPrice DOUBLE,
  markPrice  DOUBLE,
  pnl        DOUBLE,
  roe        DOUBLE,
  amount     DOUBLE,
  leverage   DOUBLE,
  event_time TIMESTAMP,
  ingest_time TIMESTAMP,
  unique_key STRING,
  dt         DATE
) USING DELTA
PARTITIONED BY (dt)
""")

# =========================
# (B) Bronze 로드 & 변환
# =========================
bronze = spark.table(BRONZE)

silver_df = (
    bronze
      .select(
        col("uid"),
        col("symbol"),
        col("entryPrice"),
        col("markPrice"),
        col("pnl"),
        col("roe"),
        col("amount"),
        col("leverage"),
        col("event_time"),
        col("ingest_time"),
        col("unique_key"),
        col("dt")
      )
      .dropDuplicates(["unique_key"])
      .repartition("dt")
)

# =========================
# (C) Delta MERGE
# =========================
target = DeltaTable.forName(spark, SILVER)
(target.alias("t")
  .merge(silver_df.alias("s"), "t.unique_key = s.unique_key AND t.dt = s.dt")
  .whenMatchedUpdateAll()
  .whenNotMatchedInsertAll()
  .execute())

print(f"Silver transform complete: upserted into {SILVER}")
